# Export MAQS site data

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = "/gws/nopw/j04/duicv/yuansun/"

In [2]:
start_time = pd.Timestamp('2022-01-01 00:00:00')
end_time = pd.Timestamp('2022-12-31 23:00:00')
station_name = 'maqs'

In [6]:
ds_check = xr.open_dataset(f'{home_path}dap.ceda.ac.uk/badc/osca/data/manchester/OSCA_MAQS_Sonic_Windmaster/2022/maqs-sonic-1_maqs_202205_mean-winds_Ratified_v2.1.nc')
ds_check

<xarray.Dataset> Size: 2MB
Dimensions:                 (time: 44640, latitude: 1, longitude: 1)
Coordinates:
  * time                    (time) datetime64[ns] 357kB 2022-05-01 ... 2022-0...
  * latitude                (latitude) float32 4B 53.46
  * longitude               (longitude) float32 4B -2.214
Data variables:
    day_of_year             (time) float32 179kB ...
    year                    (time) int16 89kB ...
    month                   (time) int16 89kB ...
    day                     (time) int16 89kB ...
    hour                    (time) int16 89kB ...
    minute                  (time) int16 89kB ...
    second                  (time) float64 357kB ...
    wind_speed              (time) float32 179kB ...
    qc_flag_wind_speed      (time) int8 45kB ...
    wind_speed_of_gust      (time) float32 179kB ...
    wind_from_direction     (time) float32 179kB ...
    qc_flag_wind_direction  (time) int8 45kB ...
Attributes: (12/39)
    Conventions:                          CF-1.6, NCAS-AMF-1.1
    source:                               maqs-sonic-1
    instrument_manufacturer:              Gill
    instrument_model:                     Windmaster
    instrument_serial_number:             180602
    instrument_software:                  unknown
    ...                                   ...
    geospatial_bounds:                    53.456636N -2.214244E
    platform_altitude:                    50 m
    location_keywords:                    MAQS, Supersite, Firs, Fallowfield
    amf_vocabularies_release:             https://github.com/ncasuk/AMF_CVs/r...
    history:                               Acquired May 2022 and Data process...
    comment:                              Measurement Height 7m above ground ...

# Observation

In [17]:
# temperature and relative humidity data
obs_path = f'{home_path}dap.ceda.ac.uk/badc/osca/data/manchester/OSCA_Manc_FIDAS_Met/2022/'
df_obs_tas = pd.DataFrame()
df_obs_rh = pd.DataFrame()
for m in range(1, 13):
    month_str = str(m).zfill(2)
    month_filename = f'{obs_path}maqs-fidas-1_maqs_2022{m:02d}_surface-met_Ratified_v2.1.nc' # monthly data
    ds_obs_month = xr.open_dataset(month_filename)
    ds_obs_sel_month = ds_obs_month.sel(time =slice(start_time, end_time))
    ds_obs_tas_month = ds_obs_sel_month['air_temperature'].rename('TAS')-273.15
    ds_obs_rh_month = ds_obs_sel_month['relative_humidity'].rename('RH')
    df_obs_tas_month = ds_obs_tas_month.to_dataframe().reset_index()
    df_obs_rh_month = ds_obs_rh_month.to_dataframe().reset_index()
    if df_obs_tas.empty:
        df_obs_tas = df_obs_tas_month
    else:
        df_obs_tas = pd.concat([df_obs_tas, df_obs_tas_month], ignore_index=True)    
    if df_obs_rh.empty:    
        df_obs_rh = df_obs_rh_month
    else:
        df_obs_rh = pd.concat([df_obs_rh, df_obs_rh_month], ignore_index=True)    

In [16]:
# wind speed data
obs_path2 = f'{home_path}dap.ceda.ac.uk/badc/osca/data/manchester/OSCA_MAQS_Sonic_Windmaster/2022/'
df_obs_wind = pd.DataFrame()
for m in range(1, 13):
    month_str = str(m).zfill(2)
    month_filename = f'{obs_path2}maqs-sonic-1_maqs_2022{m:02d}_mean-winds_Ratified_v2.1.nc' # monthly data
    ds_obs_month = xr.open_dataset(month_filename)
    ds_obs_sel_month = ds_obs_month.sel(time =slice(start_time, end_time))
    ds_obs_wind_month = ds_obs_sel_month['wind_speed'].rename('WIND')
    df_obs_wind_month = ds_obs_wind_month.to_dataframe().reset_index()
    if df_obs_wind.empty:
        df_obs_wind = df_obs_wind_month
    else:
        df_obs_wind = pd.concat([df_obs_wind, df_obs_wind_month], ignore_index=True)    

In [14]:
# precipitation data
df_rain = pd.read_csv(f'{home_path}manchester/supersite/hourly_data/final_supersite_merged.csv')
df_rain.rename(columns={'Datetime': 'time'}, inplace=True)
df_rain['time'] = pd.to_datetime(df_rain['time'])
df_rain_2022 = df_rain[(df_rain['time'] >= start_time) & (df_rain['time'] <= end_time)][['time', 'RAIN']].reset_index(drop=True)
df_rain_2022.head()

,time,RAIN
0,2022-01-01 00:00:00,0.0
1,2022-01-01 01:00:00,0.0
2,2022-01-01 02:00:00,0.0
3,2022-01-01 03:00:00,0.0
4,2022-01-01 04:00:00,0.0


In [18]:
df_obs = df_obs_tas.merge(df_obs_rh, on='time').merge(df_obs_wind, on='time').merge(df_rain_2022, on='time')
df_obs['time'] = pd.to_datetime(df_obs['time'])
df_obs = df_obs.set_index('time')
df_hourly = df_obs.resample('h').mean().reset_index()
df_hourly.to_csv(f'./data_for_figure/{station_name}_obs.csv', index=False)
df_hourly

,time,TAS,RH,WIND,RAIN
0,2022-01-01 01:00:00,13.134216,89.523003,0.652544,0.000000
1,2022-01-01 02:00:00,13.588715,89.239998,0.612789,0.000000
2,2022-01-01 03:00:00,13.797913,85.312897,0.869000,0.000000
3,2022-01-01 04:00:00,13.699921,83.943802,0.862053,0.000000
4,2022-01-01 05:00:00,13.283600,84.348297,0.949357,0.000000
...,...,...,...,...,...
8754,2022-12-31 19:00:00,9.751709,98.954002,0.820125,0.000163
8755,2022-12-31 20:00:00,9.498718,98.684303,1.556379,0.000321
8756,2022-12-31 21:00:00,9.163605,99.051697,1.409964,0.000266
8757,2022-12-31 22:00:00,8.489105,98.005203,1.205719,0.000120


In [4]:
lat = ds_obs_month['latitude'][0].values
lon = ds_obs_month['longitude'][0].values
print(lat, lon)

53.456635 -2.214244


# GM_NOAH_SLUCM

In [21]:
ds_sim_sample = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf/wrfout_d01_2021-12-25_00:00:00.nc')
model_lat = ds_sim_sample['XLAT'][0, :, 0].values
model_lon = ds_sim_sample['XLONG'][0, 0, :].values
lat_idx = np.abs(model_lat - lat).argmin()
lon_idx = np.abs(model_lon - lon).argmin()
print(lat_idx, lon_idx)

NameError: name 'lat' is not defined

In [22]:
ds_sim_sample

<xarray.Dataset> Size: 1GB
Dimensions:                (Time: 120, south_north: 45, west_east: 57,
                            bottom_top: 43, bottom_top_stag: 44,
                            soil_layers_stag: 4, west_east_stag: 58,
                            south_north_stag: 46, seed_dim_stag: 8)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 1MB ...
    XLONG                  (Time, south_north, west_east) float32 1MB ...
    XTIME                  (Time) datetime64[ns] 960B ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 1MB ...
    XLONG_U                (Time, south_north, west_east_stag) float32 1MB ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 1MB ...
    XLONG_V                (Time, south_north_stag, west_east) float32 1MB ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag, seed_dim_stag
Data variables: (12/204)
    Times                  (Time) |S19 2kB ...
    LU_INDEX               (Time, south_north, west_east) float32 1MB ...
    ZNU                    (Time, bottom_top) float32 21kB ...
    ZNW                    (Time, bottom_top_stag) float32 21kB ...
    ZS                     (Time, soil_layers_stag) float32 2kB ...
    DZS                    (Time, soil_layers_stag) float32 2kB ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 1MB ...
    PC                     (Time, south_north, west_east) float32 1MB ...
    LANDMASK               (Time, south_north, west_east) float32 1MB ...
    LAKEMASK               (Time, south_north, west_east) float32 1MB ...
    SST                    (Time, south_north, west_east) float32 1MB ...
    SST_INPUT              (Time, south_north, west_east) float32 1MB ...
Attributes: (12/136)
    TITLE:                            OUTPUT FROM WRF V4.7.0 MODEL
    START_DATE:                      2021-12-25_00:00:00
    SIMULATION_START_DATE:           2021-12-25_00:00:00
    WEST-EAST_GRID_DIMENSION:        58
    SOUTH-NORTH_GRID_DIMENSION:      46
    BOTTOM-TOP_GRID_DIMENSION:       44
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

In [ ]:
lat_idx = 17
lon_idx = 31